In [1]:
!pip install tabulate

In [2]:
import sqlite3
import csv
import pandas as pd
from tabulate import tabulate

# Connecting to the database
vincon = sqlite3.connect('coffeeshop.db')
vincon.execute("PRAGMA foreign_keys = ON")
vincur = vincon.cursor()


In [3]:
customerdf = pd.read_csv("customer_level_data.csv")
itemdf = pd.read_csv("item_level_data.csv")
transactiondf = pd.read_csv("transaction_level_data.csv")

In [4]:
customerdf.head()

,customer_number,app_install_time,gender,marital_status,date_of_birth,referral_as_source_of_app_install
0,1,2018-06-29 20:40:12,MALE,NaN,2018-06-05,Not_through_referral
1,2,2018-07-03 19:11:13,FEMALE,NaN,1998-12-08,Not_through_referral
2,3,2018-07-21 12:03:39,FEMALE,NaN,1997-11-19,Installed_through_referral
3,4,2018-07-23 18:22:37,MALE,NaN,2018-01-11,Not_through_referral
4,5,2018-08-03 01:40:45,MALE,NaN,1982-08-19,Not_through_referral


In [5]:
customerdf.shape

(10767, 6)

In [6]:
customerdf.drop("marital_status", axis=1, inplace=True)
customerdf.head()

,customer_number,app_install_time,gender,date_of_birth,referral_as_source_of_app_install
0,1,2018-06-29 20:40:12,MALE,2018-06-05,Not_through_referral
1,2,2018-07-03 19:11:13,FEMALE,1998-12-08,Not_through_referral
2,3,2018-07-21 12:03:39,FEMALE,1997-11-19,Installed_through_referral
3,4,2018-07-23 18:22:37,MALE,2018-01-11,Not_through_referral
4,5,2018-08-03 01:40:45,MALE,1982-08-19,Not_through_referral


In [7]:
itemdf.head()

,transaction_number,customer_number,item_name,sale_amount
0,4271,400,FRAPPE,203.0
1,17539,1781,FOOD92,222.5
2,8963,2444,FOOD6,332.0
3,19087,6952,FOOD132,322.5
4,5139,7786,TOPPING SYRUPS,196.0


In [8]:
itemdf.shape

(17972, 4)

In [9]:
transactiondf.head()

,transaction_number,customer_number,transaction_time,store_name,sale_amount_including_loyalty_points_used,loyalty_points_used_by_customer,offer_used_by_customer_to_earn_loyalty_points
0,463,3050,2018-06-01 15:42:43,MUMBAI STORE 333,150.5,0.0,NaN
1,465,10736,2018-06-01 15:43:28,KANCHIPURAM STORE 72,199.5,0.0,NaN
2,467,3815,2018-06-01 17:34:04,CHENNAI STORE 1093,118.0,0.0,NaN
3,469,8590,2018-06-01 17:52:11,DHANBAD STORE 817,72.5,0.0,NaN
4,471,3315,2018-06-01 17:56:01,MADURAI STORE 54,177.0,0.0,NaN


In [10]:
transactiondf.shape

(17164, 7)

In [11]:
# creating customers table and loading the data
try:
    vincur.execute("""CREATE TABLE customers(
    customer_number    INTEGER   PRIMARY KEY, 
    app_install_time  TEXT,
    gender   TEXT,
    date_of_birth     TEXT,
    referral_as_source_of_app_install  TEXT
    );""")
    
    customerdf.to_sql(name='customers',con=vincon,if_exists='append',index=False)
    vincon.commit()
    
except Exception as e:
    vincon.rollback()
    print('Error:',e)
    

In [12]:
vincur.execute("""
SELECT * FROM customers LIMIT(5)
""")
vin_customer = vincur.fetchall()
headers_cust = ['customer_number', 'app_install_time', 'gender', 'date_of_birth', 'referral_as_source_of_app_install']
print(tabulate(vin_customer, headers_cust))

  customer_number  app_install_time     gender    date_of_birth    referral_as_source_of_app_install
-----------------  -------------------  --------  ---------------  -----------------------------------
                1  2018-06-29 20:40:12  MALE      2018-06-05       Not_through_referral
                2  2018-07-03 19:11:13  FEMALE    1998-12-08       Not_through_referral
                3  2018-07-21 12:03:39  FEMALE    1997-11-19       Installed_through_referral
                4  2018-07-23 18:22:37  MALE      2018-01-11       Not_through_referral
                5  2018-08-03 01:40:45  MALE      1982-08-19       Not_through_referral


In [13]:
# creating transactions table and loading the data
try:
    vincur.execute("""CREATE TABLE transactions (
    transaction_number INTEGER PRIMARY KEY,
    customer_number INTEGER,
    transaction_time TEXT,
    store_name TEXT,
    sale_amount_including_loyalty_points_used INTEGER,
    loyalty_points_used_by_customer INTEGER,
    offer_used_by_customer_to_earn_loyalty_points TEXT,
    FOREIGN KEY(customer_number) REFERENCES customers(customer_number)
    );""")
    
    transactiondf.to_sql(name='transactions',con=vincon,if_exists='append',index=False)
    vincon.commit()
    
except Exception as e:
    vincon.rollback()
    print(e)


In [14]:
vincur.execute("""
SELECT * FROM transactions LIMIT(5)
""")
vin_trans = vincur.fetchall()
headers_trans = ['transaction_number','customer_number', 'transaction_time', 'store_name', 'sale_amount', 'loyalty_points','offer_used']
print(tabulate(vin_trans, headers_trans))


  transaction_number    customer_number  transaction_time     store_name              sale_amount    loyalty_points  offer_used
--------------------  -----------------  -------------------  --------------------  -------------  ----------------  ------------
                 463               3050  2018-06-01 15:42:43  MUMBAI STORE 333              150.5                 0
                 465              10736  2018-06-01 15:43:28  KANCHIPURAM STORE 72          199.5                 0
                 467               3815  2018-06-01 17:34:04  CHENNAI STORE 1093            118                   0
                 469               8590  2018-06-01 17:52:11  DHANBAD STORE 817              72.5                 0
                 471               3315  2018-06-01 17:56:01  MADURAI STORE 54              177                   0


In [15]:
# creating items table and loading the data
try:
    vincur.execute("""CREATE TABLE items (
    transaction_number INTEGER,
    customer_number INTEGER,
    item_name TEXT,
    sale_amount INTEGER,
    FOREIGN KEY(transaction_number) REFERENCES transactions(transaction_number),
    FOREIGN KEY(customer_number) REFERENCES customers(customer_number)
    );""")
    
    itemdf.to_sql(name='items',con=vincon,if_exists='append',index=False)
    vincon.commit()
    
except Exception as e:
    vincon.rollback()
    print(e)


In [18]:
vincur.execute("""
SELECT * FROM items LIMIT(5)
""")
vin_items = vincur.fetchall()
headers_items = ['transaction_number','customer_number', 'item_name', 'sale_amount']
print(tabulate(vin_items, headers_items))


  transaction_number    customer_number  item_name         sale_amount
--------------------  -----------------  --------------  -------------
                4271                400  FRAPPE                  203
               17539               1781  FOOD92                  222.5
                8963               2444  FOOD6                   332
               19087               6952  FOOD132                 322.5
                5139               7786  TOPPING SYRUPS          196


In [16]:
vincur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
queryview = vincur.fetchall()
print('Tables : ', queryview)

Tables :  [('customers',), ('transactions',), ('items',)]
